# RALD-LARD MACHINES

## Imports

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from raldlard.machine import rald_lard
from raldlard.stacked_machine import stacked_rarld_lard
from raldlard.tools import integer_to_rald_lard_input,outputs_to_integers

## Parameters

In [ ]:
VERBOSE = True

## Utils

In [ ]:
def run_simple_machine(instructions,input_,verbose):
    model = rald_lard(instructions = instructions,max_steps=1000)

    if verbose:
        print("Model instructions")
        print(model)
        print("-------------------------------------------------")

    output = model.run(input_,verbose=verbose)

    if verbose:
        print("-------------------------------------------------\n")
        print(f"Input: {input_} <-----> {outputs_to_integers(input_)}")

        print(f"Output: {output} <-----> {outputs_to_integers(output)}")

## RALD-LARD machines and mono RALD-LARD machines

A line of instruction specify an action that the machine should perform. 

The machine either *Delete* or *Add* the repeated character on/to the *Left* or the *Right* of a given *Input*. <br>
An instruction is the represented by the list [Delete or Add,Left or Right, Input] <br>
To keep things simple, we will use the following notations : Delete = "D", Add = "A", Left = "L",Right = "R". The input will be represented by its id. The instructions are performed in a loop. The machine halts when a Delete action fails. For instance, deleting on the Right on the input "CCCCO" will stop a machine.

In [ ]:
run_simple_machine(instructions=[['D','L',0],
                               ['D','L',0],
                               ['A','L',0]],
                input_ = ["OCCC"],
                verbose = VERBOSE)

### Compute n+1

In [ ]:
run_simple_machine(instructions=[['A','R',0],
                               ['D','L',0]],
                input_ = ["OCCC"],
                verbose = VERBOSE)

### Compute n-1

Only works with integers. If n=0 then it returns 0.

In [ ]:
run_simple_machine(instructions=[['D','R',0],
                               ['D','L',0]],
                input_ = ["OCCC"],
                verbose = VERBOSE)

### compute n//2

In [ ]:
run_simple_machine(instructions=[['D','R',0],
                               ['D','R',0],
                               ['A','L',0]],
                input_ = ["OCCCCC"],
                verbose = VERBOSE)

### Compute 2n

In [ ]:
run_simple_machine(instructions=[['D','R',0],
                               ['A','L',0],
                               ['A','L',0]],
                input_ = ["OCCCCC"],
                verbose = VERBOSE)

### Check parity of a number
For an input ${OCC..C}_n$ the output it $CO$ if n is odd and O if n is even

In [ ]:
run_simple_machine(instructions=[['D','R',0],
                               ['A','L',0],
                               ['D','R',0],
                               ['D','L',0]],
                input_ = ["OCCCC"],
                verbose = VERBOSE)

In [ ]:
run_simple_machine(instructions=[['D','R',0],
                               ['A','L',0],
                               ['D','R',0],
                               ['D','L',0]],
                input_ = ["OCCCCC"],
                verbose = VERBOSE)

### Compute n+m

In [ ]:
run_simple_machine(instructions=[['D','R',1],
                               ['A','R',0]],
                input_ = ["OCCCCC","OCCCC"],
                verbose = VERBOSE)

### Compute n-m
if n<m then the output is 0

In [ ]:
run_simple_machine(instructions=[['D','R',1],
                               ['D','R',0]],
                input_ = ["OCCCCC","OCCCC"],
                verbose = VERBOSE)

In [ ]:
run_simple_machine(instructions=[['D','R',1],
                               ['D','R',0]],
                input_ = ["OCC","OCCCC"],
                verbose = VERBOSE)

### Compare n and m
if both outputs are equals to 0 then the 2 numbers are the same
if one of the output is equal to 0 and the second one if something else then, the input whose output is 0 is the smallest.

In [ ]:
run_simple_machine(instructions=[['D','R',0],
                               ['A','L',0],
                               ['D','R',1],
                               ['D','L',0]],
                input_ = ["OCCCCC","OCCCCC"],
                verbose = VERBOSE)

In [ ]:
run_simple_machine(instructions=[['D','R',0],
                               ['A','L',0],
                               ['D','R',1],
                               ['D','L',0]],
                input_ = ["OCCCCC","OCCCCCC"],
                verbose = VERBOSE)

In [ ]:
run_simple_machine(instructions=[['D','R',0],
                               ['A','L',0],
                               ['D','R',1],
                               ['D','L',0]],
                input_ = ["OCCCC","OCCC"],
                verbose = VERBOSE)

## Stacked RALD-LARD models

Here is an example of a stacked RALD-LARD model computing $2^n$. 

In [ ]:
model_0 = rald_lard(instructions = [['A','R',0],['D','L',0]]) ## compute n+1

model_1 = rald_lard(instructions = [['D','R',0],['D','L',0]]) ## compute n-1
model_2 = rald_lard(instructions = [['D','R',1],['A','L',1],['A','L',1]])  ## compute 2n right

model_3 = rald_lard(instructions = [['D','R',0],['D','L',0]]) ## compute n-1
model_4 = rald_lard(instructions = [['D','L',1],['A','R',1],['A','R',1]])  ## compute 2n left


In [ ]:
stacked_model = stacked_rarld_lard()

stacked_model.add_model(model_0,0)
stacked_model.add_model(model_1,1)
stacked_model.add_model(model_2,2)
stacked_model.add_model(model_3,3)
stacked_model.add_model(model_4,4)

stacked_model.add_transition(0,1,[['D','R',0],['A','R',0]])
stacked_model.add_transition(1,2,[['D','R',0],['A','R',0]])
stacked_model.add_transition(2,3,[['D','R',0],['A','R',0]])
stacked_model.add_transition(3,4,[['D','R',0],['A','R',0]])
stacked_model.add_transition(4,1,[['D','R',0],['A','R',0]])

stacked_model.set_entry_point(0)

In [ ]:
if VERBOSE:
    print(stacked_model)

In [ ]:
outputs = stacked_model.run(['OCCCCC','OC'],verbose=VERBOSE)

In [ ]:
if VERBOSE:
    print(outputs_to_integers(outputs))

In [ ]:
for n in range(0,15+1):
    output_ = stacked_model.run([integer_to_rald_lard_input(n),'OC'],verbose=False)
    assert outputs_to_integers(output_)[1]==2**n

# End of notebook